In [ ]:
%matplotlib inline
!pip install mxnet 
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from tqdm.autonotebook import tqdm
from pathlib import Path

In [ ]:
# Define parameters
interval = 'D' #Use D or H

assert interval == 'D' 

if interval == 'D':
    prediction_length = 65 #when interval = D    
    context_length = 65
    
mnemonics = ['CON','DAI','PAH3','BMW','VOW3']
target_column = 'EndPrice'
covariate_columns = ['StartPrice', 'MinPrice', 'MaxPrice']

In [ ]:
# Define IAM role and session
import sagemaker
role = sagemaker.get_execution_role()
session = sagemaker.Session()

#Obtain container image URI for SageMaker-DeepAR algorithm, based on region
region = session.boto_region_name
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
print("Model will be trained using container image : {}".format(image_name))

#Define training data location
s3_data_key = 'dbg-stockdata/source'
s3_bucket = session.default_bucket()
s3_output_path = "s3://{}/{}/{}/output".format(s3_bucket, s3_data_key, interval)

In [ ]:
# Define IAM role and session
import sagemaker
role = sagemaker.get_execution_role()
session = sagemaker.Session()

#Obtain container image URI for SageMaker-DeepAR algorithm, based on region
region = session.boto_region_name
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
print("Model will be trained using container image : {}".format(image_name))

#Define training data location
s3_data_key = 'dbg-stockdata/source'
s3_bucket = session.default_bucket()
s3_output_path = "s3://{}/{}/{}/output".format(s3_bucket, s3_data_key, interval)

In [ ]:
import numpy as np
selected = stock_data_series.sort_index()
train_set, test_set= np.split(selected, [int(.8 *len(selected))])

In [ ]:
!pip install gluonts
import datetime
from gluonts.dataset.common import ListDataset
# Function to convert data frames containing time series data to JSON serialized data that DeepAR works with
def json_serialize(data, start, end, target_column, covariate_columns, interval):
    timeseries = {}

    for i, col in enumerate(data.columns):
        metric = col[col.find('-')+1:]
        stock = col[:col.find('-')]
        if metric == target_column:
            if stock in timeseries.keys():
                timeseries[stock]["target"] = data.iloc[:,i][start:end]
            else:
                timeseries[stock] = {}
                timeseries[stock]["start"] = str(pd.Timestamp(datetime.datetime.strptime(str(start), "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S"), freq = interval))
                timeseries[stock]["target"] = data.iloc[:,i][start:end]            
            print("Time series for {} added".format(stock))
        elif metric in covariate_columns:
            if stock in timeseries.keys():
                if "dynamic_feat" in timeseries[stock]:
                    dynamic_feat = timeseries[stock]["dynamic_feat"]
                    dynamic_feat.append(data.iloc[:,i][start:end])
                else:
                    dynamic_feat = []
                    dynamic_feat.append(data.iloc[:,i][start:end])
                    timeseries[stock]["dynamic_feat"] = dynamic_feat
            else:
                timeseries[stock] = {}
                dynamic_feat = []
                dynamic_feat.append(data.iloc[:,i])
                timeseries[stock]["dynamic_feat"] = dynamic_feat            
            print("Dynamic Feature - {} for {} added".format(metric, stock))
        else:
            pass

    json_data = 
        {
            "start": ts["start"],
            "target": ts["target"].tolist(),  
            "dynamic_feat": [feat.tolist() for feat in ts["dynamic_feat"]]
        }
        for ts in timeseries.values()
    
    return json_data



In [ ]:
train_start_dt = train_set.index[0]
train_end_dt = train_set.index[-1]   
test_start_dt = test_set.index[0]
test_end_dt = test_set.index[-1]   

train_json = json_serialize(train_set, train_start_dt, train_end_dt, target_column, covariate_columns, interval)
test_json = json_serialize(test_set, test_start_dt, test_end_dt, target_column, covariate_columns, interval)
with open('train.json', 'w') as train_file:
    json.dump(train_json, train_file)
with open('test.json', 'w') as test_file:
    json.dump(test_json, test_file)